In [ ]:
import numpy as np
import pandas as pd

In [ ]:
temp_df = pd.read_csv('IMDB Dataset.csv')

In [ ]:
df = temp_df.iloc[:10000]

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [ ]:
df['review'] = df['review'].apply(remove_tags)

In [ ]:
df['review'] = df['review'].apply(lambda x:x.lower())

In [ ]:
from nltk.corpus import stopwords
sw_list = stopwords.words('english')
df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [ ]:
df['review']

In [ ]:
import gensim

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [ ]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [ ]:
model.build_vocab(story)

In [ ]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
len(model.wv.index_to_key)

In [ ]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [ ]:
document_vector(df['review'].values[0])

In [ ]:
from tqdm import tqdm

In [ ]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

In [ ]:
X = np.array(X)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)